# MDP - Markov Decision Process

In [1]:
from mdp import *
from notebook import psource, pseudocode, plot_pomdp_utility

## Classe MDP

O que é necessário para se definir um MDP? 
- conjunto de estados, ações, estado inicial, modelo de transição e função de recompensa.

In [2]:
# visualização da classe MDP presente no arquivo mdp.py
psource(MDP)

### Exemplo de aplicação 

Estados A e B possuem ações X, Y disponíveis. As probabilidades são mostradas acompanhando as arestas. A classe MDP será usada como base para classe CustomMDP. Matrizes de transição são codificadas abaixo.
<img src="mdp-a.png">

In [3]:
# Transition Matrix as nested dict. State -> Actions in state -> List of (Probability, State) tuples
t = {
    "A": {
            "X": [(0.3, "A"), (0.7, "B")],
            "Y": [(1.0, "A")]
         },
    "B": {
            "X": {(0.8, "End"), (0.2, "B")},
            "Y": {(1.0, "A")}
         },
    "End": {}
}

init = "A"

terminals = ["End"]

rewards = {
    "A": 5,
    "B": -10,
    "End": 100
}

In [4]:
class CustomMDP(MDP):
    def __init__(self, init, terminals, transition_matrix, reward = None, gamma=.9):
        # All possible actions.
        actlist = []
        for state in transition_matrix.keys():
            actlist.extend(transition_matrix[state])
        actlist = list(set(actlist))
        MDP.__init__(self, init, actlist, terminals, transition_matrix, reward, gamma=gamma)

    def T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else: 
            return self.t[state][action]

Classe é instanciada com os parâmetros da figura.

In [5]:
our_mdp = CustomMDP(init, terminals, t, rewards, gamma=.9)

## GRID MDP

O MDP foi definido. Agora é hora de utilizá-lo para representar uma grid como da figura **Fig 17.1** do livro AIMA. 
O ambiente é _totalmente observável_ de modo que o agente sempre sabe sua localização no mundo. 

In [6]:
# visualização da classe GridMDP presente no arquivo mdp.py
psource(GridMDP)

In [7]:
sequential_decision_environment

## _VALUE ITERATION_

Agora que vimos como representar MDPs, vamos tentar resolvê-los. Nosso objetivo final é obter uma política ideal. Começamos examinando a Iteração de Valor e uma visualização que deve nos ajudar a entendê-la melhor. Começamos calculando Valor/Utilidade para cada um dos estados. O valor de cada estado é a soma esperada das recompensas futuras descontadas, dado que começamos naquele estado e seguimos uma política específica $\pi$. O valor ou a utilidade de um estado é dado por

$$U(s)=R(s)+\gamma\max_{a\epsilon A(s)}\sum_{s'} P(s'\ |\ s,a)U(s')$$

Essa é a equação de Bellman. O algoritmo Iteração de Valor (**Fig. 17.4** no livro) baseia-se em encontrar soluções desta Equação. A intuição da Iteração de Valor funciona porque os valores se propagam através do espaço de estados por meio de atualizações locais.

In [8]:
# visualização da função no arquivo mdp.py
psource(value_iteration)

In [9]:
value_iteration(sequential_decision_environment)

{(0, 1): 0.3984432178350045,
 (1, 2): 0.649585681261095,
 (2, 1): 0.48644001739269643,
 (0, 0): 0.2962883154554812,
 (3, 1): -1.0,
 (2, 0): 0.3447542300124158,
 (3, 0): 0.12987274656746342,
 (0, 2): 0.5093943765842497,
 (2, 2): 0.7953620878466678,
 (1, 0): 0.25386699846479516,
 (3, 2): 1.0}

## Visualização do _VALUE ITERATION_

Para ilustrar que os valores se propagam para fora dos estados, vamos criar uma visualização simples. Estaremos usando uma versão modificada da função value_iteration que armazenará U ao longo do tempo. Também removeremos o parâmetro épsilon e adicionaremos o número de iterações que desejamos.

In [10]:
def value_iteration_instru(mdp, iterations=20):
    U_over_time = []
    U1 = {s: 0 for s in mdp.states}
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    for _ in range(iterations):
        U = U1.copy()
        for s in mdp.states:
            U1[s] = R(s) + gamma * max([sum([p * U[s1] for (p, s1) in T(s, a)])
                                        for a in mdp.actions(s)])
        U_over_time.append(U)
    return U_over_time

A seguir, definimos uma função para criar a visualização a partir das utilidades retornadas por **value_iteration_instru**.

In [11]:
columns = 4
rows = 3
U_over_time = value_iteration_instru(sequential_decision_environment)

In [12]:
%matplotlib inline
from notebook import make_plot_grid_step_function

plot_grid_step = make_plot_grid_step_function(columns, rows, U_over_time)

In [22]:
import ipywidgets as widgets
from IPython.display import display
from notebook import make_visualize

iteration_slider = widgets.IntSlider(min=1, max=15, step=1, value=0)
w=widgets.interactive(plot_grid_step,iteration=iteration_slider)
display(w)

visualize_callback = make_visualize(iteration_slider)

visualize_button = widgets.ToggleButton(description = "Visualize", value = False)
time_select = widgets.ToggleButtons(description='Extra Delay:',options=['0', '0.1', '0.2', '0.5', '0.7', '1.0'])
a = widgets.interactive(visualize_callback, visualize=visualize_button, time_step=time_select)
display(a)

interactive(children=(IntSlider(value=1, description='iteration', max=15, min=1), Output()), _dom_classes=('wi…

interactive(children=(ToggleButton(value=False, description='Visualize'), ToggleButtons(description='Extra Del…

Mova o slide acima para observar como as utilidades mudam entre as iterações.O **Botão Visualize** animará automaticamente, sendo possível definir o atraso de tempo desejado de visualização (**Extra Delay**).

## _POLICY ITERATION_

A iteração de valor converge para a política ótima muito antes de estimar com precisão a função de utilidade.
Se uma ação for claramente melhor do que todas as outras, então a magnitude exata das utilidades nos estados envolvidos não necessita ser precisa. O algoritmo de iteração de políticas funciona com base nesse insight.
O algoritmo executa duas etapas fundamentais:

* **Avaliação da política**: Dada uma política _&#960;&#7522;_, calcule _U&#7522; = U(&#960;&#7522;)_, a utilidade de cada estado se _&#960;&#7522;_ fosse executado.
* **Melhoria da política**: Calcule uma nova política _&#960;&#7522;&#8330;&#8321;_ usando a previsão de uma etapa com base nos valores de utilidade calculados.

O algoritmo termina quando a etapa de melhoria da política não produz alterações nas utilidades.
Consulte a **Figura 17.6** do livro para ver como isso é uma melhoria em relação à iteração de valor.
Agora temos uma versão simplificada da equação de Bellman:

$$U_i(s) = R(s) + \gamma \sum_{s'}P(s'\ |\ s, \pi_i(s))U_i(s')$$

In [16]:
psource(expected_utility)

In [17]:
psource(policy_iteration)

In [18]:
psource(policy_evaluation)

Veja agora como resolver o **`sequential_decision_environment`** usando `policy_iteration`.

In [19]:
policy_iteration(sequential_decision_environment)

{(0, 1): (0, 1),
 (1, 2): (1, 0),
 (2, 1): (0, 1),
 (0, 0): (0, 1),
 (3, 1): None,
 (2, 0): (0, 1),
 (3, 0): (-1, 0),
 (0, 2): (1, 0),
 (2, 2): (1, 0),
 (1, 0): (1, 0),
 (3, 2): None}

## Problemas de Decisão Sequencial

De posse das ferramentas necessárias para resolver MDPs, vamos ver como os Problemas de Decisão Sequencial podem ser resolvidos passo a passo e como algumas ferramentas integradas na classe GridMDP nos ajudam a analisar melhor o problema em questão.

Considere o mundo da gride da **Figura 17.1** do livro.
![título](grid_mdp.jpg)

<br>Este é o ambiente do nosso agente. Assumimos por enquanto que o ambiente é _totalmente observável_, para que o agente sempre saiba onde ele está. Assumimos também que as transições são **Markovianas**, ou seja, a probabilidade de atingir o estado $s'$ a partir do estado $s$ depende apenas de $s$ e não do histórico de estados anteriores. Quase todos os problemas de decisão estocástica podem ser reformulados como um Processo de Decisão de Markov apenas ajustando a definição de um _estado_ para aquele problema específico.

<br>
No entanto, as ações do nosso agente neste ambiente não são confiáveis. Em outras palavras, o movimento do nosso agente é estocástico.

<br><br>
Mais especificamente, o agente pode -
* mover-se corretamente na direção pretendida com uma probabilidade de _0,8_,
* mover $90^\circ$ para a direita da direção pretendida com uma probabilidade de 0,1
* mover $90^\circ$ para a esquerda da direção pretendida com uma probabilidade de 0,1

<br><br>
O agente permanece parado se bater em uma parede.
![título](grid_mdp_agent.jpg)


In [25]:
psource(GridMDP.T)

Para definir completamente o ambiente de tarefas, precisamos especificar a função de utilidade para o agente.
Esta é a função que dá ao agente uma estimativa aproximada de quão bom é estar em um determinado estado, ou quanta _recompensa_ um agente recebe por estar nesse estado. O agente então tenta maximizar a recompensa que recebe.
Como o problema de decisão é sequencial, a função de utilidade dependerá de uma sequência de estados e não de um único estado. Por enquanto, simplesmente estipulamos que em cada estado $s$, o agente recebe uma recompensa finita $R(s)$.

Para qualquer estado, as ações que o agente pode realizar são codificadas conforme abaixo:
- Mover para cima: (0, 1)
- Mover para baixo: (0, -1)
- Mover para a esquerda: (-1, 0)
- Mover para a direita: (1, 0)
- Não faça nada: `None`

Agora nos perguntamos o que seria uma solução válida para o problema. Não podemos ter sequências de ação fixas porque o ambiente é estocástico e podemos acabar num estado indesejável. Portanto, uma solução deve especificar o que o agente deve fazer para _qualquer_ estado que o agente possa atingir.
<br>
Tal solução é conhecida como **política** e geralmente é denotada por $\pi$.
<br>
A **política ótima** é a política que produz a maior utilidade esperada e geralmente é denotada por $\pi^*$.
<br>
A classe `GridMDP` possui um método útil `to_arrows` que gera uma gride mostrando a direção que o agente deve se mover, dada uma política. Usaremos isso mais tarde para entender melhor as propriedades do ambiente.

In [26]:
# Este método codifica diretamente as ações que o agente pode realizar (descritas acima) para caracteres 
# que representam setas e as mostra em formato de gride para fins de visualização humana. 
# Ele converte a política recebida de um `dictionary` em uma grid usando o método `to_grid`.

psource(GridMDP.to_arrows)

In [27]:
psource(GridMDP.to_grid)

### Agora que temos todas as ferramentas necessárias e um bom entendimento do agente e do ambiente, consideramos alguns casos e vemos como o agente deve se comportar em cada caso.

### Caso 1
---
R(s) = -0.04 em todos os estados, exceto estados terminais

In [28]:
# Note that this environment is also initialized in mdp.py by default
sequential_decision_environment = GridMDP([[-0.04, -0.04, -0.04, +1],
                                           [-0.04, None, -0.04, -1],
                                           [-0.04, -0.04, -0.04, -0.04]],
                                          terminals=[(3, 2), (3, 1)])

Usaremos a função `best_policy` para encontrar a melhor política para este ambiente. Mas, como você pode ver, `best_policy` também requer uma função de utilidade. Já sabemos que a função de utilidade pode ser encontrada por `value_iteration`. Portanto, nossa melhor política é:

In [29]:
pi = best_policy(sequential_decision_environment, value_iteration(sequential_decision_environment, .001))

Agora podemos usar o método `to_arrows` para ver como nosso agente deve escolher suas ações no ambiente.

In [30]:
from utils import print_table
print_table(sequential_decision_environment.to_arrows(pi))

>   >      >   .
^   None   ^   .
^   >      ^   <


Este é exatamente o resultado que esperávamos
<br>
![título](-0.04.jpg)
<br>
Observe que, como o custo de dar um passo é relativamente pequeno comparado com a penalidade por terminar em `(4, 2)` por acidente, a política ótima é conservadora.
No estado `(3, 1)`, recomenda-se seguir o caminho mais longo, em vez de seguir o caminho mais curto e arriscar obter uma grande recompensa negativa de -1 em `(4, 2)`.

### Caso 2
---
R(s) = -0.4 em todos os estados, exceto estados terminais

In [31]:
sequential_decision_environment = GridMDP([[-0.4, -0.4, -0.4, +1],
                                           [-0.4, None, -0.4, -1],
                                           [-0.4, -0.4, -0.4, -0.4]],
                                          terminals=[(3, 2), (3, 1)])

In [32]:
pi = best_policy(sequential_decision_environment, value_iteration(sequential_decision_environment, .001))
from utils import print_table
print_table(sequential_decision_environment.to_arrows(pi))

>   >      >   .
^   None   ^   .
^   >      ^   <


Como a recompensa para cada estado é agora mais negativa, a vida é certamente mais desagradável.
O agente segue o caminho mais curto para o estado +1 e está disposto a correr o risco de cair no estado -1 por acidente.

### Caso 3
---
R(s) = -4 em todos os estados, exceto estados terminais

In [33]:
sequential_decision_environment = GridMDP([[-4, -4, -4, +1],
                                           [-4, None, -4, -1],
                                           [-4, -4, -4, -4]],
                                          terminals=[(3, 2), (3, 1)])

In [34]:
pi = best_policy(sequential_decision_environment, value_iteration(sequential_decision_environment, .001))
from utils import print_table
print_table(sequential_decision_environment.to_arrows(pi))

>   >      >   .
^   None   >   .
>   >      >   ^


A recompensa em termos de vida para cada estado é agora menor do que o terminal menos recompensador. A vida é tão dolorosa que o agente se dirige para a saída mais próxima, pois mesmo a pior saída é menos dolorosa do que qualquer estado de vida.

### Caso 4
---
R(s) = 4 em todos os estados, exceto estados terminais

In [35]:
sequential_decision_environment = GridMDP([[4, 4, 4, +1],
                                           [4, None, 4, -1],
                                           [4, 4, 4, 4]],
                                          terminals=[(3, 2), (3, 1)])

In [36]:
pi = best_policy(sequential_decision_environment, value_iteration(sequential_decision_environment, .001))
from utils import print_table
print_table(sequential_decision_environment.to_arrows(pi))

>   >      <   .
>   None   <   .
>   >      >   v


Nesse caso, a saída esperada é:
![title](4.jpg)
<br>
Como a vida é positivamente agradável, o agente evita ambas as saídas.
Mesmo que o resultado que obtemos não seja exatamente o que desejamos, definitivamente não está errado.
O cenário aqui pode ser qualquer coisa, alcançar um estado terminal, pois esta é a única maneira de o agente maximizar sua recompensa (a recompensa total tende ao infinito), e o programa faz exatamente isso.
<br>
Atualmente, a classe GridMDP não oferece suporte a um marcador explícito para uma ação "faça o que quiser" ou uma condição "não me importo".
No entanto, você pode estender a classe para fazer isso.